In [2]:
from stable_baselines3 import PPO
import gymnasium as gym
from gymnasium.envs.registration import register
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
checkpoint_callback = CheckpointCallback(
  save_freq=3125,
  save_path="./checkpoint/",
  name_prefix="ppo_FIR"
)
register(
    id="five_in_roll-v0",
    entry_point="fir_env:FiveInRowEnv"
)


In [3]:
check_env(gym.make("five_in_roll-v0"))

In [3]:
env = DummyVecEnv([lambda :Monitor(gym.make("five_in_roll-v0")) for _ in range(10)])
model = PPO(
    "CnnPolicy", 
    env,
    device="cuda", 
    verbose=1,
    n_steps=512,
    batch_size=512,
    n_epochs=4,
    gamma=0.94,
    learning_rate=5e-4,
    tensorboard_log="logs",
)

Using cuda device


In [4]:
model.learn(total_timesteps=1e6//2,callback=[checkpoint_callback])

Logging to logs\PPO_8


----------------------------------
| rollout/           |           |
|    ep_len_mean     | 118       |
|    ep_rew_mean     | -2.55e+06 |
| time/              |           |
|    fps             | 22        |
|    iterations      | 1         |
|    time_elapsed    | 230       |
|    total_timesteps | 5120      |
----------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 128           |
|    ep_rew_mean          | -2.93e+06     |
| time/                   |               |
|    fps                  | 23            |
|    iterations           | 2             |
|    time_elapsed         | 433           |
|    total_timesteps      | 10240         |
| train/                  |               |
|    approx_kl            | 0.00022054338 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -42.6         |
|    explained_variance   | 0     

KeyboardInterrupt: 

In [ ]:
model.save("./model/FIR.zip")

# 测试

In [ ]:
model = model.load('./checkpoint/ppo_FIR_12500_steps.zip')
for i in range(512):
    state,info = env.reset()
    done = False 
    score = 0
    while not done:
        action, _ = model.predict(observation=state)
        obs,reward,done,Terminated,info = env.step(action)
        score += reward
        env.render()
        if done:break
        input()
env.close()

In [ ]:
env.render()